In [2]:
import plotly as py
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
from google.colab import output
output.enable_custom_widget_manager()
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [5]:
!pip install scikit-learn

In [6]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, KernelPCA

In [9]:
!pip install eurostat
import eurostat
import json
import requests

In [13]:
!pip install plotly pandas eurostat

In [17]:
#zadanie 1
import pandas as pd
import eurostat

df = eurostat.get_data_df('une_rt_m')

geo_col = [col for col in df.columns if 'geo' in col][0]
df = df.rename(columns={geo_col: 'geo'})

df = df[(df['s_adj'] == 'SA') & (df['sex'] == 'T') & (df['age'] == 'TOTAL')]
df = df.drop(columns=['s_adj', 'sex', 'age', 'unit'])

latest_month = [col for col in df.columns if col.startswith('20')][-1]

df_latest = df[['geo', latest_month]].copy()
df_latest.columns = ['country_code', 'unemployment_rate']

country_code_map = {
    'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria', 'HR': 'Croatia', 'CY': 'Cyprus',
    'CZ': 'Czech Republic', 'DK': 'Denmark', 'EE': 'Estonia', 'FI': 'Finland', 'FR': 'France',
    'DE': 'Germany', 'EL': 'Greece', 'HU': 'Hungary', 'IE': 'Ireland', 'IT': 'Italy',
    'LV': 'Latvia', 'LT': 'Lithuania', 'LU': 'Luxembourg', 'MT': 'Malta', 'NL': 'Netherlands',
    'PL': 'Poland', 'PT': 'Portugal', 'RO': 'Romania', 'SK': 'Slovakia', 'SI': 'Slovenia',
    'ES': 'Spain', 'SE': 'Sweden', 'NO': 'Norway', 'IS': 'Iceland', 'CH': 'Switzerland'
}

df_latest['country'] = df_latest['country_code'].map(country_code_map)
df_latest = df_latest.dropna(subset=['country'])
df_latest = df_latest.sort_values(by='unemployment_rate', ascending=False)

print(df_latest)


    country_code  unemployment_rate         country
882           SE              489.0          Sweden
877           NL              387.0     Netherlands
865           FI              260.0         Finland
869           IE              119.0         Ireland
754           FI                9.1         Finland
771           SE                8.5          Sweden
758           IE                4.1         Ireland
766           NL                3.8     Netherlands
740           AT                NaN         Austria
742           BE                NaN         Belgium
743           BG                NaN        Bulgaria
744           CH                NaN     Switzerland
745           CY                NaN          Cyprus
746           CZ                NaN  Czech Republic
747           DE                NaN         Germany
748           DK                NaN         Denmark
750           EE                NaN         Estonia
751           EL                NaN          Greece
752         

In [18]:
#zadanie 2
from sklearn.datasets import load_wine

wine = load_wine()
feature_names = wine.feature_names
feature_names

wine_df = pd.DataFrame(np.hstack([wine.data, np.expand_dims(wine.target, axis=1)]), columns=wine.feature_names + ['target'])

print(wine.data.shape)
#99%
pca = PCA(n_components=.99)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_1.shape)
#95%
pca = PCA(n_components=.95)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_2 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_2.shape)
#90%
pca = PCA(n_components=.90)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_3 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_3.shape)

(178, 13)
(178, 1)
(178, 1)
(178, 1)


zadanie 3

In [19]:
pca = PCA(n_components=13)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
pca.explained_variance_ratio_.round(3)

array([0.998, 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   ])

zadaanie 4

In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(wine.data, wine.target)

feature_importances = pd.Series(rf_model.feature_importances_, index=wine.feature_names)
feature_importances_sorted = feature_importances.sort_values(ascending=False)

cumulative_importance = feature_importances_sorted.cumsum()

threshold_99 = cumulative_importance[cumulative_importance >= 0.99].index[0]
threshold_95 = cumulative_importance[cumulative_importance >= 0.95].index[0]
threshold_90 = cumulative_importance[cumulative_importance >= 0.90].index[0]

selector_99 = SelectFromModel(rf_model, threshold=feature_importances_sorted[threshold_99], prefit=True)
X_selected_99 = selector_99.transform(wine.data)
selected_features_99 = np.array(wine.feature_names)[selector_99.get_support()]

selector_95 = SelectFromModel(rf_model, threshold=feature_importances_sorted[threshold_95], prefit=True)
X_selected_95 = selector_95.transform(wine.data)
selected_features_95 = np.array(wine.feature_names)[selector_95.get_support()]

selector_90 = SelectFromModel(rf_model, threshold=feature_importances_sorted[threshold_90], prefit=True)
X_selected_90 = selector_90.transform(wine.data)
selected_features_90 = np.array(wine.feature_names)[selector_90.get_support()]

print(f"Features selected for cumulative importance >= 99% ({X_selected_99.shape[1]} features):")
print(selected_features_99)
print(f"Features selected for cumulative importance >= 95% ({X_selected_95.shape[1]} features):")
print(selected_features_95)
print(f"Features selected for cumulative importance >= 90% ({X_selected_90.shape[1]} features):")
print(selected_features_90)
print("\nComparison of number of features/components:")
print(f"Task 2 (PCA 99% variance): {reduced_1.shape[1]} components")
print(f"Task 2 (PCA 95% variance): {reduced_2.shape[1]} components")
print(f"Task 2 (PCA 90% variance): {reduced_3.shape[1]} components")
print(f"Task 4 (RF 99% cumulative importance): {X_selected_99.shape[1]} features")
print(f"Task 4 (RF 95% cumulative importance): {X_selected_95.shape[1]} features")
print(f"Task 4 (RF 90% cumulative importance): {X_selected_90.shape[1]} features")



Features selected for cumulative importance >= 99% (12 features):
['alcohol' 'malic_acid' 'ash' 'alcalinity_of_ash' 'magnesium'
 'total_phenols' 'flavanoids' 'proanthocyanins' 'color_intensity' 'hue'
 'od280/od315_of_diluted_wines' 'proline']
Features selected for cumulative importance >= 95% (10 features):
['alcohol' 'malic_acid' 'alcalinity_of_ash' 'magnesium' 'total_phenols'
 'flavanoids' 'color_intensity' 'hue' 'od280/od315_of_diluted_wines'
 'proline']
Features selected for cumulative importance >= 90% (8 features):
['alcohol' 'malic_acid' 'total_phenols' 'flavanoids' 'color_intensity'
 'hue' 'od280/od315_of_diluted_wines' 'proline']

Comparison of number of features/components:
Task 2 (PCA 99% variance): 13 components
Task 2 (PCA 95% variance): 1 components
Task 2 (PCA 90% variance): 1 components
Task 4 (RF 99% cumulative importance): 12 features
Task 4 (RF 95% cumulative importance): 10 features
Task 4 (RF 90% cumulative importance): 8 features
